In [1]:
import warnings 
warnings.filterwarnings('ignore')

from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def predict_next_sen(input_text, num_predictions=5):
    input_text_with_mask = input_text + ' ' + ' '.join(['[MASK]'] * num_predictions)
    input_ids = tokenizer.encode(input_text_with_mask, return_tensors='pt')
    
 
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits

    predicted_sentence = input_text
    for i in range(num_predictions):
       
        mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
       
        predicted_token_id = predictions[0, mask_token_index[0]].argmax(dim=-1)
        predicted_token = tokenizer.decode(predicted_token_id)
        
       
        input_text_with_mask = input_text_with_mask.replace('[MASK]', predicted_token, 1)
        predicted_sentence += ' ' + predicted_token
        
       
        input_ids = tokenizer.encode(input_text_with_mask, return_tensors='pt')

    return predicted_sentence



BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [3]:
input_text = input()
predicted_sentence = predict_next_sen(input_text, num_predictions=5)
print(predicted_sentence)


virat kohli is
virat kohli is a a indian actor .
